<a href="https://colab.research.google.com/github/Mike-Pits/NU-Stazhirovki-Qazaq/blob/main/Pipeline_%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Инициализация моделей

In [ ]:
# тут код для скачивания моделей в папку колаба

In [ ]:
import os

# Установка переменной окружения
os.environ['MY_VAR'] = 'some_value'

In [ ]:
# Синглтон для однократной загрузки моделей
# Данный класс будет инициироваться при старте приложения
# И далее не будет требоваться время на загрузку моделей
# Каждый раз при создании экземпляра класса возвращается один объект
# Можно расширять, сколько надо, подгружая модели
# open–closed principle, OCP
class ModelsHub:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(ModelsHub, cls).__new__(cls)
            # Инициализация моделей здесь
            cls._instance.rotate_model = cls.load_rotate_model()
            cls._instance.model_b = cls.load_b_model()
        return cls._instance

    @staticmethod
    def load_rotate_model():
        # при желании пути до моделей можно брать из ENV
        # my_var_value = os.environ.get('MY_VAR')
        # model_path = '/app/model_files/rotation_model.h5'
        # return tf.keras.models.load_model(path, custom_objects=CUSTOM_OBJECTS)
        return 'Rotate model'

    @staticmethod
    def load_b_model():
        # model_path = ""
        # return YOLO(model_path)
        return 'modelb'

In [ ]:
# Пример использования
models = ModelsHub()
print(models.rotate_model)
print(models.model_b)

Rotate model
modelb


In [ ]:
# Контекст является объектом хранящим данных и конфигурации
# И изменяемым в процессе прохождения через пайплайн
class Context:
  def __init__(self, models, images, processing_params = {}, metadata = {}):
  # def __init__(self, models: ModelsHub, images: List[ImageDTO], processing_params: dict = {}, metadata: dict = {}):
    self.images = images
    self.models = models
    self.processing_params = processing_params
    self.metadata = metadata
  # может содержать методы сигнализирующие об ошибке обработки
  # а также методы агрегирующие разультат
  # напириер конечный результат после завершения обработки

In [ ]:
# пример использования хаба моделей и контекста
models = ModelsHub()
context = Context(models, [])
print(context.models.rotate_model)

# в любом месте где доступен контекст может быть вызвана модель AI

Rotate model


In [ ]:
# Класс ячейки таблицы
# подобные классы можно создать
# для любого объекта с логикой поддержки консистентности
class TableRow:
    def __init__(self, item, unit, quantity, price, amount):
        self.item = item
        self.unit = unit
        self.quantity = quantity
        self.price = price
        self.amount = amount

    @classmethod
    def create_from_dict(cls, row_data):
        required_keys = ["item", "unit", "quantity", "price", "amount"]
        if isinstance(row_data, dict) and all(key in row_data for key in required_keys):
            return cls(row_data["item"], row_data["unit"], row_data["quantity"], row_data["price"], row_data["amount"])
        else:
            print("Некорректные данные для строки таблицы")
            return None

    def to_json(self):
        return {
            "10-01 Item": self.item,
            "10-02 Unit": self.unit,
            "10-03 Quantity": self.quantity,
            "10-04 Price": self.price,
            "10-05 Amount": self.amount
        }

In [ ]:
# Data Transfer Object (DTO)
# Объект изображения включающий все методы по работе с изображением
class ImageDTO:
    # переменные которые нельзя менять через set_attr
    PROTECTED_ATTRIBUTES = ['path', 'name', 'prepared_img']

    def __init__(self, path, name):
        self.path = path
        self.name = name
        # salt.raw - может быть например numpy объект чтобы не загрудать повторно
        self.prepared_img = None
        self.receipt_number = ''
        self.receipt_date = ''
        self.items_table = []

    def _create_items_table_json(self):
        return [row.to_json() for row in self.items_table]

    # Пример метода сохраняющего путь до файла после предобработки
    def set_prepared_img(self, img_path):
       self.prepared_img = img_path

    # метод установки значения отрибутов
    def set_attr(self, attribute_name, value):
        if attribute_name in ImageDTO.PROTECTED_ATTRIBUTES:
            print(f"Ошибка: Нельзя изменять атрибут '{attribute_name}'.")
            return
        setattr(self, attribute_name, value)

    def add_table_row(self, row_data):
        row = TableRow.create_from_dict(row_data)
        if row is not None:
            self.items_table.append(row)

    # Пример метода возвращающего информацию
    # о изображении в виде json объекта
    # необходим для выводе
    def to_json(self):
       example = {
          "File Name": self.name,
          "01 Receipt Number": self.receipt_number,
          "02 Receipt Date": self.receipt_date,
          "10 Items Table": self._create_items_table_json(),
       }
       return example

In [ ]:
import json
# Пример использования
image = ImageDTO('/tmp/adfasdf', '0123.png')

print(image.path)
print(image.name)
print(image.to_json())

# добавление в объект значений переменных
image.set_attr('receipt_number', '0987')

# добавление ячеек таблицы
image.add_table_row({"item": "Item1", "unit": "pcs", "quantity": 10, "price": 5.0, "amount": 50.0})
image.add_table_row({"item": "Item1", "unit": "pcs", "quantity": 20, "price": 2.0, "amount": 40.0})

# Форматированный вывод JSON
formatted_json = json.dumps(image.to_json(), indent=2)
print(formatted_json)

/tmp/adfasdf
0123.png
{'File Name': '0123.png', '01 Receipt Number': '', '02 Receipt Date': '', '10 Items Table': []}
{
  "File Name": "0123.png",
  "01 Receipt Number": "0987",
  "02 Receipt Date": "",
  "10 Items Table": [
    {
      "10-01 Item": "Item1",
      "10-02 Unit": "pcs",
      "10-03 Quantity": 10,
      "10-04 Price": 5.0,
      "10-05 Amount": 50.0
    },
    {
      "10-01 Item": "Item1",
      "10-02 Unit": "pcs",
      "10-03 Quantity": 20,
      "10-04 Price": 2.0,
      "10-05 Amount": 40.0
    }
  ]
}


## Пример использования хаба моделей и контекста и ImageDTO

In [ ]:
# пример использования хаба моделей и контекста и ImageDTO
models = ModelsHub()
image = ImageDTO('/tmp/adfasdf', '0123.png')
context = Context(models, [image])

print(context.models.rotate_model)
print(context.images[0].to_json())


Rotate model
{'File Name': '0123.png', '01 Receipt Number': '', '02 Receipt Date': '', '10 Items Table': []}


##  Пайплайн

Класс Pipeline представляет собой последовательность обработчиков (процессоров), каждый из которых выполняет определенную задачу обработки. Эти обработчики должны быть наследниками класса BaseProcessor и реализовывать метод process. В конструкторе класса Pipeline создается список self.pipeline, содержащий экземпляры обработчиков: ImagePreprocessor, NeuralNetworkPredictor, и PostProcessor.

Метод run класса Pipeline последовательно применяет каждый процессор из списка к контексту. Каждый процессор может модифицировать ImageDTO в контексте, добавляя в него результаты своей работы или изменяя его текущее состояние. После прохождения через все процессоры, модифицированный контекст возвращается как результат работы всего пайплайна.

Эта архитектура позволяет гибко управлять процессом обработки данных, комбинируя различные процессоры и передавая контекст между ними для обмена информацией и результатами обработки.








In [ ]:
# Класс BaseProcessor служит в качестве абстрактного базового класса для всех процессоров
# в системе обработки данных.
# Он определяет обязательный метод process, который должен быть реализован во всех классах-наследниках.
# Этот метод принимает объект context, который содержит все необходимые данные и состояния, передаваемые между процессорами.
class BaseProcessor:
    def process(self, context):
        raise NotImplementedError("Each processor must implement the 'process' method.")

In [ ]:
class PostProcessor(BaseProcessor):
    def process(self, context):
        # Всегда принимает и возвращает контекст
        # внутри контекста есть все данные необходимые для обработаки
        # context.models - загруженнные модели AI
        # context.images - массив изображений ImageDTO
        return context

In [ ]:
#
class Pipeline:
    def __init__(self):
        self.pipeline = [
            # ImagePreprocessor(),
            # NeuralNetworkPredictor(),
            PostProcessor()
        ]

    def run(self, context):
        for processor in self.pipeline:
            context = processor.process(context)
        return context